In [3]:
from collections import OrderedDict
from collections import namedtuple

import numpy as np
from scipy import stats


# R precision
def r_precision(targets, predictions, max_n_predictions=500):
    # Assumes predictions are sorted by relevance
    # First, cap the number of predictions
    predictions = predictions[:max_n_predictions]

    # Calculate metric
    target_set = set(targets)
    target_count = len(target_set)
    return float(len(set(predictions[:target_count]).intersection(target_set))) / target_count

def dcg(relevant_elements, retrieved_elements, k, *args, **kwargs):
    """Compute the Discounted Cumulative Gain.
    Rewards elements being retrieved in descending order of relevance.
    \[ DCG = rel_1 + \sum_{i=2}^{|R|} \frac{rel_i}{\log_2(i + 1)} \]
    Args:
        retrieved_elements (list): List of retrieved elements
        relevant_elements (list): List of relevant elements
        k (int): 1-based index of the maximum element in retrieved_elements
        taken in the computation
    Note: The vector `retrieved_elements` is truncated at first, THEN
    deduplication is done, keeping only the first occurence of each element.
    Returns:
        DCG value
    """
    retrieved_elements = __get_unique(retrieved_elements[:k])
    relevant_elements = __get_unique(relevant_elements)
    if len(retrieved_elements) == 0 or len(relevant_elements) == 0:
        return 0.0
    # Computes an ordered vector of 1.0 and 0.0
    score = [float(el in relevant_elements) for el in retrieved_elements]
    # return score[0] + np.sum(score[1:] / np.log2(
    #     1 + np.arange(2, len(score) + 1)))
    return np.sum(score / np.log2(1 + np.arange(1, len(score) + 1)))


def ndcg(relevant_elements, retrieved_elements, k, *args, **kwargs):
    """Compute the Normalized Discounted Cumulative Gain.
    Rewards elements being retrieved in descending order of relevance.
    The metric is determined by calculating the DCG and dividing it by the
    ideal or optimal DCG in the case that all recommended tracks are relevant.
    Note:
    The ideal DCG or IDCG is on our case equal to:
    \[ IDCG = 1+\sum_{i=2}^{min(\left| G \right|, k)}\frac{1}{\log_2(i +1)}\]
    If the size of the set intersection of \( G \) and \( R \), is empty, then
    the IDCG is equal to 0. The NDCG metric is now calculated as:
    \[ NDCG = \frac{DCG}{IDCG + \delta} \]
    with \( \delta \) a (very) small constant.
    The vector `retrieved_elements` is truncated at first, THEN
    deduplication is done, keeping only the first occurence of each element.
    Args:
        retrieved_elements (list): List of retrieved elements
        relevant_elements (list): List of relevant elements
        k (int): 1-based index of the maximum element in retrieved_elements
        taken in the computation
    Returns:
        NDCG value
    """

    # TODO: When https://github.com/scikit-learn/scikit-learn/pull/9951 is
    # merged...
    idcg = dcg(
        relevant_elements, relevant_elements, min(k, len(relevant_elements)))
    if idcg == 0:
        raise ValueError("relevent_elements is empty, the metric is"
                         "not defined")
    true_dcg = dcg(relevant_elements, retrieved_elements, k)
    return true_dcg / idcg


def __get_unique(original_list):
    """Get only unique values of a list but keep the order of the first
    occurence of each element
    """
    return list(OrderedDict.fromkeys(original_list))


Metrics = namedtuple('Metrics', ['r_precision', 'ndcg', 'plex_clicks'])


# playlist extender clicks
def playlist_extender_clicks(targets, predictions, max_n_predictions=500):
    # Assumes predictions are sorted by relevance
    # First, cap the number of predictions
    predictions = predictions[:max_n_predictions]

    # Calculate metric
    i = set(predictions).intersection(set(targets))
    for index, t in enumerate(predictions):
        for track in i:
            if t == track:
                return float(int(index / 10))
    return float(max_n_predictions / 10.0 + 1)


# def compute all metrics
def get_all_metrics(targets, predictions, k):
    return Metrics(r_precision(targets, predictions, k),
                   ndcg(targets, predictions, k),
                   playlist_extender_clicks(targets, predictions, k))


MetricsSummary = namedtuple('MetricsSummary', ['mean_r_precision',
                                               'mean_ndcg',
                                               'mean_plex_clicks',
                                               'coverage'])


In [16]:
#java -jar RankLib-2.10.jar -train BigRecall-TrainingFile750-2080.txt -ranker 6 -metric2t NDCG@20 -save BigRecallTrain750-2080Model-1Trees-NDCG20-tc1-lr05-leaf5.txt -tree 1 -tc 1 -shrinkage 0.5 -leaf 5
#java -jar RankLib-2.10.jar -load BigRecallTrain750-2080Model-500Trees-NDCG20-tc1-lr05-leaf50.txt -rank BigRecallTestingFile750.txt -score BRScores750-2080Model-500Trees-NDCG20-tc1-lr05-leaf50.txt

#java -jar RankLib-2.10.jar -train BigRecall-TrainingFile750-2080.txt -ranker 6 -metric2t NDCG@20 -save BigRecallTrain750-2080Model-500Trees-NDCG20-tc1-lr05-leaf50.txt -tree 500 -tc 1 -shrinkage 0.5 -leaf 50


import os
TrainingFile='./Training/BigRecall-TrainingFile750-2080.txt'
TestingFile='./Training/BigRecallTestingFile750.txt'
trees=[500]
tcVals=[-1]
shrinkages=[0.5]
leaves= [50]


def createCommand(tree,tc,lr,leaf):
    opModelFile= './Training/ModelsAndScores/BigRecallTrain750-2080Model-'+str(tree)+'Trees-NDCG20-tc'+str(tc)+'-lr'+str(lr).replace('.','')+'-leaf'+str(leaf)+'.txt'
    trainCommand= 'java -jar ./Training/RankLib-2.10.jar -train ./Training/BigRecall-TrainingFile750-2080.txt -ranker 6 -metric2t NDCG@20 -silent -save '+ opModelFile+ ' -tree '+str(tree)+' -tc '+str(tc)+ ' -shrinkage '+str(lr)+ ' -leaf '+ str(leaf) +' -missingzero'
    #BRScores750-2080Model-1Trees-NDCG20-tc1-lr05-leaf5.txt
    opScoresFile='./Training/ModelsAndScores/BRScores750-2080Model-'+opModelFile.split('Model-')[1]
    testCommand= 'java -jar ./Training/RankLib-2.10.jar -load '+opModelFile+' -rank ./Training/BigRecallTestingFile750.txt -score '+opScoresFile
    return (opModelFile,trainCommand, opScoresFile, testCommand)
paramSweep=[]
for tree in trees:
    for tc in tcVals:
        for lr in shrinkages:
            for leaf in leaves:
                paramSweep.append(createCommand(tree,tc,lr,leaf))

In [ ]:
import multiprocessing as mp
import codecs
import os
import subprocess

def ExecuteRanklib(execTuples):
    try:
        trainCommand=execTuples[1]
        train= subprocess.check_output(trainCommand.split())
        print train
        print '----------'
        scoreCommand=execTuples[3]
        test=subprocess.check_output(scoreCommand.split())
        print test
        print '----------'
        
    except:
        print execTuples


pool = mp.Pool(processes=10)
pool.map(ExecuteRanklib, paramSweep)

Process PoolWorker-30:
Process PoolWorker-23:
Process PoolWorker-24:
Process PoolWorker-27:
Process PoolWorker-26:
Process PoolWorker-25:
Process PoolWorker-28:


('./Training/ModelsAndScores/BigRecallTrain750-2080Model-500Trees-NDCG20-tc-1-lr05-leaf50.txt', 'java -jar ./Training/RankLib-2.10.jar -train ./Training/BigRecall-TrainingFile750-2080.txt -ranker 6 -metric2t NDCG@20 -silent -save ./Training/ModelsAndScores/BigRecallTrain750-2080Model-500Trees-NDCG20-tc-1-lr05-leaf50.txt -tree 500 -tc -1 -lr 0.5 -leaf 50', './Training/ModelsAndScores/BRScores750-2080Model-500Trees-NDCG20-tc-1-lr05-leaf50.txt', 'java -jar ./Training/RankLib-2.10.jar -load ./Training/ModelsAndScores/BigRecallTrain750-2080Model-500Trees-NDCG20-tc-1-lr05-leaf50.txt -rank ./Training/BigRecallTestingFile750.txt -score ./Training/ModelsAndScores/BRScores750-2080Model-500Trees-NDCG20-tc-1-lr05-leaf50.txt')


Process PoolWorker-22:
KeyboardInterrupt
KeyboardInterrupt
Process PoolWorker-29:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap


KeyboardInterrupt: 

  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap


In [4]:
TestFile='./Training/BigRecallTestingFile750.txt'

with open(TestFile) as f:
    test = f.readlines()

PidTestTracks={}
for l in test:
    pid=l.split()[1].split(':')[1].strip()
    track=l.split('#')[1].strip()
    PidTestTracks.setdefault(pid,[]).append(track)
    
###skip
import os

Meta1Resultspath='/home/ubuntu/SpotifyChallenge/notebooks/Reranking/TestingQueryResults/Meta1/'
Meta2Resultspath='/home/ubuntu/SpotifyChallenge/notebooks/Reranking/TestingQueryResults/Meta2/'
QEPRFResultspath='/home/ubuntu/SpotifyChallenge/notebooks/Reranking/TestingQueryResults/QEPRF750/'

Meta1Files=[Meta1Resultspath+x for x in os.listdir(Meta1Resultspath)]
Meta2Files=[Meta2Resultspath+x for x in os.listdir(Meta2Resultspath)]
QEPRFFiles=[QEPRFResultspath+x for x in os.listdir(QEPRFResultspath)]

###skip
import codecs
def parseMetaFiles(path):
    playlistId=path.split('/')[-1].split('.op')[0]
    with codecs.open(path, 'r', encoding='utf-8') as f:
        lines = f.read().splitlines()
    rank=0
    resultSet=[]
    for result in lines[1:]:
        try:
            rank=rank+1
            splits=result.split('\t')
            score = splits[0]
            trackid= splits[1]
            resultSet.append((rank,trackid,score))
        except:
            print result
            return "QueryError"
    return(playlistId,resultSet)

####skip
Meta1Op=[]
err1=[]
Meta2Op=[]
err2=[]
for f in Meta1Files:
    res=parseMetaFiles(f)
    if res !="QueryError":
        Meta1Op.append(res)
    else:
        err1.append(f)
for f in Meta2Files:
    res=parseMetaFiles(f)
    if res !="QueryError":
        Meta2Op.append(res)
    else:
        err2.append(f)
        
####skip
import codecs
def QEPRFParse(path):
    playlistId=path.split('/')[-1].split('.op')[0]
    with codecs.open(path, 'r', encoding='utf-8') as f:
        lines = f.read().splitlines()
    inputQueries=lines[0].split('# query: ')[1].split()
    resultSet=[]
    pairResults= lines[1].split(' #weight(')[2].split(') )')[0].split('"  ')
    rank=0
    for result in pairResults[:-1]:
        try:
            rank=rank+1
            splits=result.split('"')
            score = splits[0].strip()
            trackid= splits[1].strip()
            resultSet.append((rank,trackid,score))
        except:
            print result
            return "QueryError"
    return(playlistId,inputQueries,resultSet)

###skip
QEPRFOp=[]
err3=[]
for f in QEPRFFiles:
    res=QEPRFParse(f)
    if res !="QueryError":
        QEPRFOp.append(res)
    else:
        err3.append(f)
        
###skip
import pickle
pidTrackMapping=pickle.load(open('./BiPartites/AllDataPidTrackListBipartite.pkl','rb'))

####skip
import pickle
import os
import codecs
from random import shuffle

pkl = os.listdir('./SplitsInformation/')

count=0
DS={}
for fpkl in pkl:
    if fpkl in ['testing25RandPid.pkl', 'testing25Pid.pkl', 'testing1Pid.pkl', 'testing100Pid.pkl', 'testing10Pid.pkl', 'testing5Pid.pkl', 'testing100RandPid.pkl']:
        testType=fpkl.replace('.pkl','')
        if 'Rand' in fpkl:
            listLen=int(fpkl.split('testing')[1].split('Rand')[0])
            qtype='Rand'
        else :
            listLen=int(fpkl.split('testing')[1].split('Pid')[0])
            qtype='Normal'
        testingPids=pickle.load(open('./SplitsInformation/'+fpkl,'rb'))
        for pid in testingPids:
            pid=str(pid)
            referenceSet=[x.replace('spotify:track:','') for x in pidTrackMapping[pid]]
            DS[pid]=(testType,qtype,listLen,referenceSet)

####skip
import pickle
import os
import codecs
from random import shuffle

pkl = os.listdir('./SplitsInformation/')
testingTitleonlyPids=[]
for fpkl in pkl:
    if fpkl =='testingOnlyTitlePid.pkl':
        testType=fpkl.replace('.pkl','')
        listLen=0
        qtype='Normal'
        testingPids=pickle.load(open('./SplitsInformation/'+fpkl,'rb'))
        for pid in testingPids:
            pid=str(pid)
            referenceSet=[x.replace('spotify:track:','') for x in pidTrackMapping[pid]]
            DS[pid]=(testType,qtype,listLen,referenceSet)
testingTitleonlyPids=[str(x) for x in testingPids]



# EXCEPTION in query 759505: IndriRunQuery.cpp(410): QueryThread::_runQuery Exception
# EXCEPTION in query 864483: IndriRunQuery.cpp(410): QueryThread::_runQuery Exception
# EXCEPTION in query 110994: IndriRunQuery.cpp(410): QueryThread::_runQuery Exception
# EXCEPTION in query 591668: IndriRunQuery.cpp(410): QueryThread::_runQuery Exception
# EXCEPTION in query 754459: IndriRunQuery.cpp(410): QueryThread::_runQuery Exception
# EXCEPTION in query 479140: IndriRunQuery.cpp(410): QueryThread::_runQuery Exception
# EXCEPTION in query 221635: IndriRunQuery.cpp(410): QueryThread::_runQuery Exception
# EXCEPTION in query 841212: IndriRunQuery.cpp(410): QueryThread::_runQuery Exception
# EXCEPTION in query 943553: IndriRunQuery.cpp(410): QueryThread::_runQuery Exception
# EXCEPTION in query 512541: IndriRunQuery.cpp(410): QueryThread::_runQuery Exception
# EXCEPTION in query 841351: IndriRunQuery.cpp(410): QueryThread::_runQuery Exception
# EXCEPTION in query 609540: IndriRunQuery.cpp(410): Q

In [15]:
from collections import defaultdict
from random import shuffle

for comb in paramSweep:
    scoresfile= comb[2]
    with open(scoresfile) as f:
        scores = f.readlines()
    
    PidTracksScores={}
    for l in scores:
        pid=l.split()[0].strip()
        trackScore=l.split()[2].strip()
        PidTracksScores.setdefault(pid,[]).append(float(trackScore))
        
    
    rerankedCandidates={}
    for pid,tracksList  in PidTestTracks.items():
        scoresList=PidTracksScores[pid]
        zippedPairs=zip(tracksList,scoresList)
        shuffle(zippedPairs)
        rerankedCandidates[pid]=[x[0] for x in sorted(zippedPairs, key=lambda x: x[1], reverse=True)]
    
    ####continue here
    evalSets=[]
    for pl in QEPRFOp:
        plId=pl[0]
        exposed=pl[1]
        candidates=rerankedCandidates[plId]
        candidates=[x for x in candidates if x not in exposed]
        refVals= DS[plId]
        testtype=refVals[0]
        orderType=refVals[1]
        exposedLen=refVals[2]
        playlist=refVals[3]
        if orderType=='Normal':
            groundTruth=playlist[exposedLen:]
        else:
            groundTruth=[x for x in playlist if x not in exposed]
        evalSets.append((groundTruth, candidates[:500], testtype, exposedLen))

    for pl in Meta2Op:
        plId=pl[0]
        if plId in testingTitleonlyPids and plId in rerankedCandidates:
            exposed=[]
            candidates=rerankedCandidates[plId]
            refVals= DS[plId]
            testtype=refVals[0]
            orderType=refVals[1]
            exposedLen=refVals[2]
            playlist=refVals[3]
            groundTruth=playlist[exposedLen:]
            evalSets.append((groundTruth, candidates[:500], testtype, exposedLen))
    ####continue here

    '''
    r_precision(targets, predictions, k),
                       ndcg(targets, predictions, k),
                       playlist_extender_clicks(targets, predictions, k)
    '''

    indivSumsCounts= defaultdict(int)
    indivSumsRecall = defaultdict(int)
    indivSumsNdcg = defaultdict(int)
    indivSumsRprec = defaultdict(int)
    indivSumsClicks = defaultdict(int)
    globalNdcg=0
    globalRprec=0
    globalClicks=0
    globalRecall=0
    
    count=0
    for evalTuple in evalSets:
        targets=evalTuple[0]
        predictions=evalTuple[1]
        testType=evalTuple[2]
        tupNdcg=ndcg(targets,predictions,500)
        tuprprec=r_precision(targets,predictions,500)
        tupClicks=playlist_extender_clicks(targets,predictions,500)
        globalNdcg+=tupNdcg
        indivSumsNdcg[testType]+=tupNdcg
        globalRprec+=tuprprec
        indivSumsRprec[testType]+=tuprprec
        globalClicks+=tupClicks
        indivSumsClicks[testType]+=tupClicks

        indivSumsCounts[testType]+=1

        recallSetSize= len(set(predictions)&set(targets))
        refSetSize=len(targets)
        recall=recallSetSize*1.0/refSetSize
        globalRecall+=recall
        indivSumsRecall[testType]+=recall

        count+=1
    for k, v in indivSumsCounts.items():
        indivSumsRecall[k]=indivSumsRecall[k]/v
        indivSumsNdcg[k]=indivSumsNdcg[k]/v
        indivSumsRprec[k]=indivSumsRprec[k]/v
        indivSumsClicks[k]=indivSumsClicks[k]/v
    
    print scoresfile , 'Recall:' , globalRecall/count,'NDCG:', globalNdcg/count, 'RPrec:', globalRprec/count,'Clicks:', globalClicks/count
    

./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc-1-lr09-leaf5.txt Recall: 0.496151328292 NDCG: 0.301458189779 RPrec: 0.128395610746 Clicks: 4.9145316253
./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc-1-lr09-leaf10.txt Recall: 0.500125251481 NDCG: 0.303755406247 RPrec: 0.129254491629 Clicks: 4.83206565252
./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc-1-lr09-leaf50.txt Recall: 0.504065314042 NDCG: 0.305168030632 RPrec: 0.128284880919 Clicks: 4.72037630104
./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc-1-lr09-leaf100.txt Recall: 0.504055750427 NDCG: 0.303409031112 RPrec: 0.126503903426 Clicks: 4.77862289832
./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc-1-lr09-leaf250.txt Recall: 0.499622453926 NDCG: 0.293808130357 RPrec: 0.117779772983 Clicks: 4.95756605284
./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc-1-lr09-leaf500.txt Recall: 0.49332748546 NDCG: 0.283637203164 

./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc1-lr001-leaf5.txt Recall: 0.29067557438 NDCG: 0.157215568317 RPrec: 0.0372009712077 Clicks: 10.0460368295
./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc1-lr001-leaf10.txt Recall: 0.289394769489 NDCG: 0.156601687689 RPrec: 0.0376528455114 Clicks: 10.1635308247
./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc1-lr001-leaf50.txt Recall: 0.292444824552 NDCG: 0.15778318612 RPrec: 0.0373786088941 Clicks: 9.94355484388
./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc1-lr001-leaf100.txt Recall: 0.294248646511 NDCG: 0.158974313151 RPrec: 0.036974201641 Clicks: 9.88090472378
./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc1-lr001-leaf250.txt Recall: 0.290479831739 NDCG: 0.157681284091 RPrec: 0.0370027374565 Clicks: 10.0304243395
./Training/ModelsAndScores/BRScores750-2080Model-100Trees-NDCG20-tc1-lr001-leaf500.txt Recall: 0.29053691402 NDCG: 0.1577295093

In [ ]:
 Recall 0.5964542020518547,
 NDCG 0.30332032798678032,
 RPrec 0.12934009424035461,
 Clicks 5.1286